In [1]:
# Previously created environment in Anaconda console with conda create -n mlops-zoomcamp-project python=3.8
!conda activate mlops-zoomcamp-project


CommandNotFoundError: Your shell has not been properly configured to use 'conda activate'.
To initialize your shell, run

    $ conda init <SHELL_NAME>

Currently supported shells are:
  - bash
  - fish
  - tcsh
  - xonsh
  - zsh
  - powershell

See 'conda init --help' for more information and options.

IMPORTANT: You may need to close and restart your shell after running 'conda init'.




In [2]:
# Sanity check that the environment is active
import os
print (os.environ['CONDA_DEFAULT_ENV'])

mlops-zoomcamp-project


In [3]:
# Question 1: MLflow version

!mlflow --version

mlflow, version 1.28.0


In [4]:
%cd mlflow

/home/usuario/mlops-zoomcamp-project/train_mlflow_prefect/mlflow


In [5]:
!dir

hpo.py	preprocess_data.py  register_model.py  train.py


In [6]:
!python preprocess_data.py --raw_data_path ../data --dest_path ../output

In [7]:
%cd ../output

/home/usuario/mlops-zoomcamp-project/train_mlflow_prefect/output


In [8]:
#Question 2: How many files saved to output folder?

import os
print(len([name for name in os.listdir('.') if os.path.isfile(name)]))

4


In [9]:
%cd ..

/home/usuario/mlops-zoomcamp-project/train_mlflow_prefect


In [10]:
# Added this in train.py right before rf = RandomForestRegressor(max_depth=10, random_state=0)
#    mlflow.set_tracking_uri("sqlite:///mlflow.db")
#    mlflow.set_experiment("nyc-taxi-experiment")
#    mlflow.sklearn.autolog()
#    with mlflow.start_run():

!python mlflow/train.py

2022/09/12 00:05:42 INFO mlflow.tracking.fluent: Experiment with name 'house-rent-experiment' does not exist. Creating a new experiment.
2022/09/12 00:05:47 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/usuario/anaconda3/envs/mlops-zoomcamp-project/lib/python3.8/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils."


In [11]:
# Question 3: How many parameters are automatically logged by MLflow?

# I executed mlflow ui --backend-store-uri sqlite:///mlflow.db to open mlflow in http://127.0.0.1:5000 and
# checked the parameters of the last run, which were 17

In [12]:
# Question 4: In addition to backend-store-uri, what else do you need to pass to properly configure the server?

# I run in Anaconda console: mlflow ui --backend-store-uri sqlite:///mlflow.db --default-artifact-root ./mlruns

In [13]:
# I enclosed the RandomForestRegressor with: with mlflow.start_run():
# and logged the parameters as well as the rmse metric at the last line of the block

!python mlflow/hpo.py

2022/09/12 00:05:59 INFO mlflow.tracking.fluent: Experiment with name 'random-forest-hyperopt' does not exist. Creating a new experiment.
100%|██████████| 50/50 [00:25<00:00,  2.00trial/s, best loss: 25462.99739058372]


In [14]:
# Question 5: What's the best validation RMSE that you got?

# 6.628

In [15]:
!python mlflow/register_model.py

2022/09/12 00:06:27 INFO mlflow.tracking.fluent: Experiment with name 'random-forest-best-models' does not exist. Creating a new experiment.
2022/09/12 00:06:30 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/usuario/anaconda3/envs/mlops-zoomcamp-project/lib/python3.8/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils."
Successfully registered model 'house-rent-regressor'.
2022/09/12 00:06:41 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: house-rent-regressor, version 1
Created version '1' of model 'house-rent-regressor'.


In [16]:
# Retrieve the run id associated with the registered best model

import mlflow
from mlflow.tracking import MlflowClient

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("house-rent-experiment")

client = MlflowClient()
run_id_best_model = client.get_latest_versions("house-rent-regressor", stages=["None"])[0].run_id
print(run_id_best_model)

40b7243af8de4ad99f402320bd80a066


In [17]:
# Question 6: What is the test RMSE of the best model?

mlflow.get_run(run_id_best_model).data.metrics['test_rmse']

25761.342517995818